## Imports

In [1]:
import scipy.io as sio
import numpy as np
from matplotlib import pyplot as plt
from math import e
from math import log
from sklearn.metrics import classification_report

## Load the Data [Images]

In [2]:
# Load Data Here

## CODE

### Classifier Class

In [3]:
sigmoid = lambda points: [(1/(1+e**(-pt))) for pt in points]
class classifier:
    def __init__(self, method, weights):
        self.method = method
        self.weights = weights
    
    def predict_probs(self, points):
        pred = [self.method([np.dot(pt, self.weights) + self.weights[-1:]]) for pt in points]
        return pred
    
    def predict_classes(self, points):
        pred = [self.method([np.dot(pt, self.weights[:-1])]) for pt in points]
        _p = []
        for pt in pred:
            if pt[0][0] < 0.5:
                _p.append(0)
            elif pt[0][0] > 0.5 and pt[0][0] < 1.01:
                _p.append(1)
        return _p

## MINIMIZATION

### (STOCHASTIC) GRADIENT DESCENT

In [4]:
def GD(X, y, _epsislon, _lambda, num_iterations, stochastic=False):
    old_val, w, i = np.zeros((13,1)), np.zeros((13,1)), 0
    same = []
    if stochastic == True:
        points = [pt for pt in X]
        classes = [cls for cls in y]
        up = len(points) // 50
        for j in range(up):
            x = np.array(X[:(up // 2)])
            y = np.array(y[:(up // 2)])
            while i < num_iterations:
                dt_p = np.dot(x, w)
                correction = np.dot(x.T, (y - sigmoid(dt_p)))
                correction = (_epsislon*correction + 2*_lambda*w)
                w += correction 
                i+=1
    else:
        while i < num_iterations:
            dt_p = np.dot(X, w)
            correction = np.dot((X.T), (y - sigmoid(dt_p)))
            correction = (_epsislon/(i+1))*correction + 2*_lambda*w
            w += correction 
            i+=1
    return w

def SGD(X, y, _epsislon, _lambda, num_iterations):
    return GD(X, y, _epsislon, _lambda, num_iterations, stochastic=True)

### LOGISTIC REGRESSION

In [5]:
def logistic_regression(data, opt_method, _epsilon, _lambda, num_iter):
    X = data['X']
    X = add_dim(X)
    y = data['y']
    weights = opt_method(X, y, _epsilon, _lambda, num_iter)
    _classifier = classifier(sigmoid, weights)
    return _classifier

### Helper Functions

In [6]:
sigmoid = lambda points: [(1/(1+e**(-pt))) for pt in points]

def test_clf(predicted, true):
    score = 0
    for i in range(len(predicted)):
        if predicted[i] == true[i]:
            score+=1
    if (score/len(predicted)) >= 0.93:
        print("SUCCESS!")
    else:
        print("FAILURE...")
    return score/len(predicted)

def add_dim(X):
    N = np.zeros((6000,13), float)    
    #for i in range(len(X)):
    for i in range(6000):
        N[i] = np.append(X[i], [1])
    return N

def add_dim2(X):
    N = np.zeros((6000,13), float)    
    #for i in range(len(X)):
    for i in range(6000):
        N[i] = np.append(X[i], [1])
    return N

def cost_fn(y, X, w):
    var  = 0
    X = add_dim(X)
    for i in range(len(y)):
        dot  = np.dot(X[i], w)
        var += -y*np.log(sigmoid(dot)) + (1+y[i])*np.log(1-dot)
    return var

## Visualizing